In [1]:
# 1. Does home-field advantage exist?
# 2. Does weather have an effect on team performance in terms of average goals scored per game?
# 3. Does weather diminish home-field advantage?

In [278]:
import pandas as pd
import json
import requests
from datetime import datetime
from datetime import timedelta

In [279]:
import mysql.connector
import config

In [280]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.pw,
    database = 'soccer_matches'
)
cursor = cnx.cursor()

In [281]:
matches = pd.read_sql("select * from matches", cnx)
matches['datetime'] = matches['datetime'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%S'))
matches.head()

,match_key,datetime,season,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,home_team_goal_count,...,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,stadium_name
0,AFC Bournemouth vs Arsenal 01-03-17,2017-01-03T19:45:00,2016,1483472700,Jan 3 2017 - 7:45pm,complete,11202,AFC Bournemouth,Arsenal,3,...,14,7,5,6,9,9,9,45,55,Vitality Stadium
1,AFC Bournemouth vs Arsenal 01-14-18,2018-01-14T13:30:00,2017,1515936600,Jan 14 2018 - 1:30pm,complete,10836,AFC Bournemouth,Arsenal,2,...,13,6,5,8,8,7,10,45,55,Vitality Stadium
2,AFC Bournemouth vs Arsenal 02-07-16,2016-02-07T13:30:00,2015,1454851800,Feb 7 2016 - 1:30pm,complete,11357,AFC Bournemouth,Arsenal,0,...,6,5,4,9,2,5,8,56,44,Vitality Stadium
3,AFC Bournemouth vs Arsenal 11-25-18,2018-11-25T13:30:00,2018,1543152600,Nov 25 2018 - 1:30pm,complete,10792,AFC Bournemouth,Arsenal,1,...,21,6,5,6,16,6,11,43,57,Vitality Stadium
4,AFC Bournemouth vs Aston Villa 08-08-15,2015-08-08T14:00:00,2015,1439042400,Aug 8 2015 - 2:00pm,complete,11155,AFC Bournemouth,Aston Villa,0,...,3,3,2,7,1,12,13,62,38,Vitality Stadium


In [282]:
cnx.commit()

In [283]:
key = 'ee86dca2621e186b5a0b4e68330f01ff'

In [284]:
# https://github.com/jokecamp/FootballData/blob/master/other/stadiums-with-GPS-coordinates.csv

stadium_coordinates = {'Emirates Stadium': (51.555, -0.108611), 'Villa Park': (52.509167, -1.884722),
                      'Ewood Park': (53.728611, -2.489167), 'Reebok Stadium': (53.580556, -2.535556),
                      'Stamford Bridge': (51.481667, -0.191111), 'Goodison Park': (53.438889, -2.966389),
                      'Craven Cottage': (51.475, -0.221667), 'Anfield': (53.431040, -2.960798),
                      'Etihad Stadium': (53.482989, -2.200292), 'Old Trafford': (53.463056, -2.291389),
                      'Sports Direct Arena': (54.975556, -1.621667), 'Carrow Road': (52.622128, 1.308653),
                      'Loftus Road': (51.509167, -0.232222), 'Britannia Stadium': (52.988333, -2.175556),
                      'Stadium of Light': (54.9144, -1.3882), 'Liberty Stadium': (51.6422, -3.9351),
                      'White Hart Lane': (51.603333, -0.065833), 'The Hawthorns': (52.509167, -1.963889),
                      'DW Stadium': (53.547778, -2.653889), 'Molineux Stadium': (52.590278, -2.130278),
                      "St. Andrew's Stadium": (52.475703, -1.868189), 'Bloomfield Road': (53.804722, -3.048056),
                      'American Express Community Stadium': (50.861822, -0.083278), 'Turf Moor': (53.789167, -2.230278),
                      'Cardiff City Stadium': (51.472778, -3.203056), 'Selhurst Park': (51.398333, -0.085556),
                      'KC Stadium': (53.746111, -0.367778), 'King Power Stadium': (52.620278, -1.142222),
                      'Riverside Stadium': (54.578333, -1.216944), 'Fratton Park': (50.796389, -1.063889),
                      'Madejski Stadium': (51.422222, -0.982778), "St Mary's Stadium": (50.905833, -1.391111),
                      'Vicarage Road': (51.649836, -0.401486), 'Boleyn Ground': (51.531944, 0.039444),
                      'Bramall Lane': (53.3703, -1.4709), 'Dean Court': (50.7352, -1.8383),
                      "The John Smith's Stadium": (53.6543, -1.7683), 'Tottenham Hotspur Stadium' : (51.6043, -0.0664),
                      'Vitality Stadium': (50.7352, -1.8383), 'University of Bolton Stadium': (53.5814, -2.5370),
                      "St. James' Park": (54.9762, -1.6218), 'Kiyan Prince Foundation Stadium': (51.5093, -0.2321),
                      'Wembley Stadium': (51.5560, -0.2795), 'London Stadium': (51.5387, 0.0166)}

# print (len(stadium_coordinates))

In [285]:
# Match Key = Liverpool-Norwich-City-August-9-2019
# Stadium Name
# Time = [YYYY]-[MM]-[DD]T[HH]:[MM]:[SS]

def get_url(match_id, coordinates_dict, stadium, time):
    match_dict = {}
    url = ('https://api.darksky.net/forecast/' + key + '/' + str(coordinates_dict[stadium][0]) +
           ',' + str(coordinates_dict[stadium][1]) + ',' + time)
    response = requests.get(url)
    match_dict[match_id] = response.json()['currently']
#     match_dict = {}
#     dt = datetime.strptime(time, '%Y-%m-%dT%H:%M:%S')
#     dt = (dt + timedelta(hours=1))
#     time = dt.strftime('%Y-%m-%dT%H:%M:%S')
#     url = ('https://api.darksky.net/forecast/' + key + '/' + str(coordinates_dict[stadium][0]) +
#            ',' + str(coordinates_dict[stadium][1]) + ',' + time)
#     print (url)
#     response = requests.get(url)
#     match_dict[match_id] = response.json()['currently']
    return match_dict

In [277]:
# x = get_url('Manchester City vs Liverpool 09-09-17', stadium_coordinates, 'Molineux Stadium', "2017-09-09T15:00:00")
# x

In [256]:
# def get_url2(match_id, coordinates_dict, stadium, time):
#     match_dict = {}
#     url = ('https://api.darksky.net/forecast/' + key + '/' + str(coordinates_dict[stadium][0]) +
#            ',' + str(coordinates_dict[stadium][1]) + ',' + time)
#     response = requests.get(url)
#     match_dict[match_id] = response.json()
#     return match_dict

In [286]:
# x = get_url2('Burnley vs Leicester City 03-16-19', stadium_coordinates, 'Turf Moor', '2019-03-16T15:00:00')
# x

In [300]:
weather_data = pd.DataFrame()
for index, row in matches[(matches['season'] == 2014) | (matches['season'] == 2015) | (matches['season'] == 2016) | (matches['season'] == 2017) | (matches['season'] == 2018)].iterrows():
    game = get_url(row['match_key'], stadium_coordinates, row['stadium_name'], row['datetime'])
    game = (pd.DataFrame.from_dict(game))
    game = game.transpose()
    weather_data = pd.concat([weather_data, game], axis =0)
weather_data

/Users/Robert/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,apparentTemperature,cloudCover,dewPoint,humidity,icon,ozone,precipAccumulation,precipIntensity,precipProbability,precipType,pressure,summary,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
AFC Bournemouth vs Arsenal 01-03-17,34.79,0.95,35.59,0.88,cloudy,NaN,NaN,0,0,NaN,1028.2,Overcast,38.83,1483472700,0,6.216,269,7.93,5.44
AFC Bournemouth vs Arsenal 01-14-18,41.25,0.75,33.31,0.67,partly-cloudy-day,NaN,NaN,0,0,NaN,1017.9,Mostly Cloudy,43.49,1515936600,1,6.216,92,4.17,4.17
AFC Bournemouth vs Arsenal 02-07-16,43.21,0.44,37.52,0.65,partly-cloudy-day,NaN,NaN,0,0,NaN,996.7,Partly Cloudy,48.75,1454851800,1,6.216,250,19.15,14.32
AFC Bournemouth vs Arsenal 11-25-18,40.59,0.78,40.15,0.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.36,1.54315e+09,0,6.216,49,24.17,12.85
AFC Bournemouth vs Aston Villa 08-08-15,76.57,0.44,56.88,0.51,partly-cloudy-day,NaN,NaN,0,0,NaN,1023.7,Partly Cloudy,76.57,1439038800,6,6.216,160,7.48,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wolverhampton Wanderers vs Newcastle United 02-11-19,34.54,0.98,34.1,0.85,cloudy,287,NaN,0,0,NaN,1030.7,Overcast,38.2,1549915200,0,10,223,6.64,4.77
Wolverhampton Wanderers vs Southampton 09-29-18,58.44,NaN,42.8,0.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.44,1.53823e+09,NaN,6.216,240,NaN,8.05
Wolverhampton Wanderers vs Tottenham Hotspur 11-03-18,51.69,0.83,47.55,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.69,1.54127e+09,0,6.216,172,24.17,12.93
Wolverhampton Wanderers vs Watford 10-20-18,58.36,0.19,50.56,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.36,1.54004e+09,2,6.216,200,NaN,7.29


In [301]:
weather_data = weather_data.drop(columns=['apparentTemperature', 'cloudCover', 'dewPoint', 'humidity', 'ozone', 'precipProbability', 
                                          'pressure', 'uvIndex', 'windBearing', 'windGust'])


In [302]:
weather_data = weather_data.fillna(0)

In [303]:
weather_data.to_csv('weather_data.csv', sep=',')

In [304]:
weather_data = pd.read_csv('weather_data.csv', index_col = False)
weather_data

,match_key,icon,precipAccumulation,precipIntensity,precipType,summary,temperature,time,visibility,windSpeed
0,AFC Bournemouth vs Arsenal 01-03-17,cloudy,0.0000,0.0000,0,Overcast,38.83,1.483473e+09,6.216,5.44
1,AFC Bournemouth vs Arsenal 01-14-18,partly-cloudy-day,0.0000,0.0000,0,Mostly Cloudy,43.49,1.515937e+09,6.216,4.17
2,AFC Bournemouth vs Arsenal 02-07-16,partly-cloudy-day,0.0000,0.0000,0,Partly Cloudy,48.75,1.454852e+09,6.216,14.32
3,AFC Bournemouth vs Arsenal 11-25-18,0,0.0000,0.0000,0,0,46.36,1.543153e+09,6.216,12.85
4,AFC Bournemouth vs Aston Villa 08-08-15,partly-cloudy-day,0.0000,0.0000,0,Partly Cloudy,76.57,1.439039e+09,6.216,4.80
...,...,...,...,...,...,...,...,...,...,...
1895,Wolverhampton Wanderers vs Newcastle United 02...,cloudy,0.0000,0.0000,0,Overcast,38.20,1.549915e+09,10.000,4.77
1896,Wolverhampton Wanderers vs Southampton 09-29-18,0,0.0000,0.0000,0,0,58.44,1.538226e+09,6.216,8.05
1897,Wolverhampton Wanderers vs Tottenham Hotspur 1...,0,0.0000,0.0000,0,0,51.69,1.541274e+09,6.216,12.93
1898,Wolverhampton Wanderers vs Watford 10-20-18,0,0.0000,0.0000,0,0,58.36,1.540040e+09,6.216,7.29


In [305]:
cursor.execute("""DROP TABLE weather""")

In [306]:
create_query = """
CREATE TABLE weather
    (match_key varchar(150), icon varchar(150), precipAccumulation float, precipIntensity float,
    precipType varchar(150), summary varchar(150), temperature float, time int, visibility float, windSpeed float, PRIMARY KEY (match_key))
"""

In [307]:
cursor.execute(create_query)

In [311]:
cnx.commit()

In [309]:
def insert_to_table(df):
    global cnx, cursor
    debug = True
    # creating column list for insertion
    cols = ", ".join([str(i) for i in df.columns.tolist()])
    # Insert DataFrame records one by one.
    for i, row in df.iterrows():
        # Convert datetime obj to string
        sql = "INSERT INTO weather (" + cols + ") VALUES (" + "%s," * (len(row) - 1) + "%s)"
        if debug:
            print(sql)
            print(tuple(row))
        cursor.execute(sql, tuple(row))
        # the connection is not auto-committed by default, so we must commit to save our changes
        cnx.commit()

In [310]:
insert_to_table(weather_data)

INSERT INTO weather (match_key, icon, precipAccumulation, precipIntensity, precipType, summary, temperature, time, visibility, windSpeed) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
('AFC Bournemouth vs Arsenal 01-03-17', 'cloudy', 0.0, 0.0, '0', 'Overcast', 38.83, 1483472700.0, 6.216, 5.44)
INSERT INTO weather (match_key, icon, precipAccumulation, precipIntensity, precipType, summary, temperature, time, visibility, windSpeed) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
('AFC Bournemouth vs Arsenal 01-14-18', 'partly-cloudy-day', 0.0, 0.0, '0', 'Mostly Cloudy', 43.49, 1515936600.0, 6.216, 4.17)
INSERT INTO weather (match_key, icon, precipAccumulation, precipIntensity, precipType, summary, temperature, time, visibility, windSpeed) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
('AFC Bournemouth vs Arsenal 02-07-16', 'partly-cloudy-day', 0.0, 0.0, '0', 'Partly Cloudy', 48.75, 1454851800.0, 6.216, 14.32)
INSERT INTO weather (match_key, icon, precipAccumulation, precipIntensity, precipType, summary, tempera

In [215]:
# team_list = ['arsenal', 'astonvilla', 'birmingham', 'blackburn', 'blackpool', 'bolton', 'bournemouth', 'brighton', 
#              'burnley', 'cardiff', 'chelsea', 'crystalpalace', 'everton', 'fulham', 'huddersfield', 'hull', 
#              'leicester', 'liverpool', 'mancity', 'manunited', 'middlesbrough', 'newcastle', 'norwich',
#              'portsmouth', 'qpr', 'reading', 'sheffieldunited', 'southampton', 'stoke', 'sunderland', 'swansea',
#              'tottenham', 'watford', 'westbrom', 'westham', 'wigan', 'wolves']

In [213]:
# def get_team_data(team_name):
#     team = pd.read_csv('team_data/' + team_name + '.csv')
#     team = team.loc[team['To'] >= '2010-08-14']
#     return team

In [214]:
# for team in team_list:
#     x = get_team_data(team)
# x.head()